<a href="https://colab.research.google.com/github/OlegV12/NaturalLangProcessing/blob/Lesson_12/Lesson_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🤗 Transfer learning и Huggingface

Благодаря тому, что большие сети обладают хорошей обобщающей способностью, мы можем брать предобученные сети и использовать их в своих целях. Кроме того, сети можно дообучать на своих задачах.

Очень пополулярной библиотекой, хостингом для хранения и инференса (запуска) моделей является 🤗 **[Huggingface](https://huggingface.co/)**.



## mBART huggingface
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.4 MB/s 
     |████████████████████████████████| 1.3 MB 38.1 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 596 kB 68.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#### Загружаем модели

In [2]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

In [3]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


#### Суммаризация текста

In [ ]:
print(model)

In [5]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [6]:
!pip install razdel networkx pymorphy2[fast] nltk rouge==0.3.1 summa lexrank sumy
!pip install --upgrade datasets razdel spacy networkx seaborn scipy tqdm tensorflow-text
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 69 kB 5.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 10.1 MB 32.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.2 MB 25.1 MB/s 
     |████████████████████████████████| 371 kB 71.7 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54412 sha256=9c083b872e3314fa1af9d89dcdd518b957689f1e45230ea9b0297638faac2030
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=8f9ca5490aa949bae99

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [7]:
# по сути данные те же можно пользоваться любым способом загрузки
from datasets import load_dataset

dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [9]:
train_records = read_gazeta_records("gazeta_train.txt")

In [11]:
train_records[0]['text']

'«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожидать серьезного замедления оттока капитала на непонятно каких причинах», — гов

In [12]:
input_ids = tokenizer.prepare_seq2seq_batch(
    train_records[0]['text'],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [13]:
print(summary)

Минэкономразвития прогнозирует замедление оттока капитала из России в 2011 году до $80 млрд. Это один из самых высоких за последние 20 лет. Больше ушло лишь в 2008 году на фоне разрастания финансового кризиса и российско-грузинской войны — $133,7 млрд.


In [15]:
train_records[0]['summary']

'В 2011 году из России уйдет $80 млрд, считают в Минэкономразвития. Менее месяца назад Центробанк давал оценку $70 млрд, повысив первоначальный прогноз вдвое. Отток капитала из страны усиливается из-за кризиса в Европе, а в декабре российским компаниям выплачивать внешние долги. На движение капитала повлияли и выборы: несмотря на их предсказуемость, бизнес хочет ясности с перестановками в правительстве.'

In [19]:
input_ids = tokenizer.prepare_seq2seq_batch(
    train_records[1]['text'],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [20]:
print(summary)

Российский офис Google возглавила Юлия Соловьева, экс-директор московского офиса международной консалтинговой компании Alvarez & Marsal. В новой должности она будет отвечать за стратегическое развитие компании в России, «на одном из крупнейших европейских интернет-рынков». По мнению экспертов, более активных действий от Google можно было бы ожидать, если бы в качестве руководителя был нанят технологический специалист или маркетолог.


In [21]:
print(train_records[1]['summary'])

Юлия Соловьева, экс-директор холдинга «Профмедиа», возглавила российский офис Google. Поиск американская интернет-корпорация вела полгода. Медиаменеджер с опытом работы в международных компаниях найдет общий язык с акционерами, но прорыва можно было ждать от технического специалиста или маркетолога, говорят аналитики.


Результат суммаризации вполне достойный. 